In [1]:
import pandas as pd


CSV_FILES = [ "Triana_Solar_pv.csv", "Longi_Solar_pv.csv", "Qcells_pv.csv"]

# df_Triana = pd.read_csv("Triana_Solar_pv.csv")
# df_longi_pv = pd.read_csv("Longi_Solar_pv.csv")
# df_qcells_pv = pd.read_csv("Qcells_pv.csv")

dfs = []
for csv_file in CSV_FILES:
    df = pd.read_csv(csv_file)
    dfs.append(df)
 
df = pd.concat(dfs, ignore_index=True)


In [ ]:
import pandas as pd
import uuid


def bool_or_null(val):
    if pd.isna(val):
        return "NULL"
    if str(val).strip().upper() in ["1", "Y", "YES", "TRUE"]:
        return "TRUE"
    return "FALSE"

def num_or_null(val):
    return "NULL" if pd.isna(val) else val

def int_or_null(val):
    return "NULL" if pd.isna(val) else int(val)

sql_statements = []
ids = [] 
for _, row in df.iterrows():
    new_id = uuid.uuid4()
    ids.append(new_id)
    sql = f"""
    INSERT INTO pv_module_technology (
        id,
        cec_name,
        technology,
        bifacial,
        power_stc,
        power_ptc,
        area,
        length,
        width,
        cells_in_series,
        short_circuit_current,
        open_circuit_voltage,
        current_max_power,
        voltage_max_power,
        temp_coeff_short_circuit_current,
        temp_coeff_open_circuit_voltage,
        temp_coeff_max_power,
        nominal_cell_temperature,
        building_integrated
    ) VALUES (
        '{new_id}',
        '{row["ModelName"]}',
        '{row["Technology"]}',
        {bool_or_null(row["Bifacial"])},
        {num_or_null(row["STC"])},
        NULL,
        {num_or_null(row["A_c"])},
        {num_or_null(row["Length"])},
        {num_or_null(row["Width"])},
        {int_or_null(row["N_s"])},
        {num_or_null(row["I_sc_ref"])},
        {num_or_null(row["V_oc_ref"])},
        {num_or_null(row["I_mp_ref"])},
        {num_or_null(row["V_mp_ref"])},
        {num_or_null(row["alpha_sc"])},
        {num_or_null(row["beta_oc"])},
        {num_or_null(row["gamma_r"])},
        {num_or_null(row["T_NOCT"])},
        {bool_or_null(row["BIPV"])}
    );
    """
    sql_statements.append(sql)

with open("seed_pv_module_technology_inserts.sql", "w") as f:
    f.write("\n".join(sql_statements))

delete_sql = f"""
    DELETE FROM pv_module_technology
    WHERE id IN ({', '.join(f"'{i}'" for i in ids)});
    """

with open("seed_pv_module_technology_delete.sql", "w") as f:
    f.write(delete_sql)

print("SQL files generated")


SQL file generated: pv_module_technology_inserts.sql
